# Load-Dependent Example 3: Multi-Class Load Dependence with PS Scheduling

This example demonstrates load-dependent queues with multiple job classes and PS (Processor Sharing) scheduling.

**Features:**
- Two closed classes: Class1 (4 jobs), Class2 (2 jobs)
- Three stations: Delay + two PS queues with load dependence
- Each queue has 3 servers with load dependence: min(total_jobs, servers)
- Compares load-independent model vs load-dependent model
- Demonstrates different solver methods: CTMC, NC, MVA

In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Model parameters
N = 4  # number of jobs in Class1
c = 3  # number of servers per queue

# Create load-independent model for comparison
model = Network('model')
node1 = Delay(model, 'Delay')
node2 = Queue(model, 'Queue1', SchedStrategy.PS)
node3 = Queue(model, 'Queue2', SchedStrategy.PS)

# Create job classes
jobclass1 = ClosedClass(model, 'Class1', N, node1, 0)
jobclass2 = ClosedClass(model, 'Class2', N // 2, node1, 0)  # N/2 = 2 jobs

# Set service times
node1.set_service(jobclass1, Exp.fit_mean(1.0))  # mean = 1.0
node1.set_service(jobclass2, Exp.fit_mean(2.0))  # mean = 2.0

node2.set_service(jobclass1, Exp.fit_mean(1.5))  # mean = 1.5
node2.set_service(jobclass2, Exp.fit_mean(2.5))  # mean = 2.5
node2.set_number_of_servers(c)

node3.set_service(jobclass1, Exp.fit_mean(3.5))  # mean = 3.5
node3.set_service(jobclass2, Exp.fit_mean(4.5))  # mean = 4.5
node3.set_number_of_servers(c)

# Create routing matrix (serial routing for each class)
P = model.init_routing_matrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)
model.link(P)

In [3]:
# Solve load-independent model with MVA
solver_mva = MVA(model, method='exact')
msT = solver_mva.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.5467 0.5467 1.0000  1.0000 0.5467 0.5467
  Delay   Class2 0.3694 0.3694 2.0000  2.0000 0.1847 0.1847
 Queue1   Class1 0.8564 0.7452 1.5664  1.5664 0.5467 0.5467
 Queue1   Class2 0.4808 0.7452 2.6027  2.6027 0.1847 0.1847
 Queue2   Class1 2.5969 0.9921 4.7498  4.7498 0.5467 0.5467
 Queue2   Class2 1.1498 0.9921 6.2244  6.2244 0.1847 0.1847


In [4]:
# Create load-dependent model
ldmodel = Network('ldmodel')
node1 = Delay(ldmodel, 'Delay')
node2 = Queue(ldmodel, 'Queue1', SchedStrategy.PS)
node3 = Queue(ldmodel, 'Queue2', SchedStrategy.PS)

# Create job classes
jobclass1 = ClosedClass(ldmodel, 'Class1', N, node1, 0)
jobclass2 = ClosedClass(ldmodel, 'Class2', N // 2, node1, 0)

# Set service times (same as load-independent model)
node1.set_service(jobclass1, Exp.fit_mean(1.0))
node1.set_service(jobclass2, Exp.fit_mean(2.0))

node2.set_service(jobclass1, Exp.fit_mean(1.5))
node2.set_service(jobclass2, Exp.fit_mean(2.5))

node3.set_service(jobclass1, Exp.fit_mean(3.5))
node3.set_service(jobclass2, Exp.fit_mean(4.5))

# Set load dependence: min(total_jobs, c) servers
total_jobs = N + N // 2  # 4 + 2 = 6 total jobs
alpha = np.minimum(np.arange(1, total_jobs + 1), c)  # min(1:6, 3) = [1,2,3,3,3,3]
node2.set_load_dependence(alpha)
node3.set_load_dependence(alpha)

# Create routing matrix
P = ldmodel.init_routing_matrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node3, 1.0)
P.set(jobclass1, jobclass1, node3, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)
ldmodel.link(P)

In [5]:
# Solve with CTMC (exact solution)
solver_ctmc = CTMC(ldmodel)
lldAvgTableCTMC = solver_ctmc.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.5467 0.5467 1.0000  1.0000 0.5467 0.5467
  Delay   Class2 0.3694 0.3694 2.0000  2.0000 0.1847 0.1847
 Queue1   Class1 0.8564 0.4739 1.5664  1.5664 0.5467 0.5467
 Queue1   Class2 0.4808 0.2713 2.6027  2.6027 0.1847 0.1847
 Queue2   Class1 2.5969 0.6941 4.7498  4.7498 0.5467 0.5467
 Queue2   Class2 1.1498 0.2980 6.2244  6.2244 0.1847 0.1847


In [6]:
# Solve with NC (Normalizing Constant) - Exact method
solver_nc_exact = NC(ldmodel, method='exact')
lldAvgTableNC = solver_nc_exact.avg_table()

Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
  Delay   Class1 0.1935 0.1935  1.0000  1.0000 0.1935 0.1935
  Delay   Class2 0.1399 0.1399  2.0000  2.0000 0.0700 0.0700
 Queue1   Class1 0.5274 0.2902  2.7264  2.7264 0.1935 0.1935
 Queue1   Class2 0.3074 0.1749  4.3938  4.3938 0.0700 0.0700
 Queue2   Class1 3.2791 0.6771 16.9504 16.9504 0.1935 0.1935
 Queue2   Class2 1.5527 0.3148 22.1934 22.1934 0.0700 0.0700


In [7]:
# Solve with NC - Reduction heuristic (RD) method
solver_nc_rd = NC(ldmodel, method='rd')
lldAvgTableRD = solver_nc_rd.avg_table()

Empty DataFrame
Columns: [Station, JobClass, QLen, Util, RespT, ResidT, ArvR, Tput]
Index: []


In [8]:
# Solve with NC - NRP and NRL methods
solver_nc_nrp = NC(ldmodel, method='nrp')
lldAvgTableNRP = solver_nc_nrp.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.7365 0.2857 1.0000  1.0000 0.7365 0.7365
  Delay   Class2 0.4876 0.4444 2.0000  2.0000 0.2438 0.2438
 Queue1   Class1 0.9791 0.4286 1.3293  1.3293 0.7365 0.7365
 Queue1   Class2 0.5401 0.5556 2.2156  2.2156 0.2438 0.2438
 Queue2   Class1 2.2845 1.0000 3.1018  3.1018 0.7365 0.7365
 Queue2   Class2 0.9723 1.0000 3.9880  3.9880 0.2438 0.2438


/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/pfqn/ncld.py:178: ComplexWarning: Casting complex values to real discards the imaginary part
  L = np.atleast_2d(np.asarray(L, dtype=float))


In [9]:

solver_nc_nrl = NC(ldmodel, method='nrl')
lldAvgTableNRL = solver_nc_nrl.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.7365 0.2857 1.0000  1.0000 0.7365 0.7365
  Delay   Class2 0.4876 0.4444 2.0000  2.0000 0.2438 0.2438
 Queue1   Class1 0.9790 0.4286 1.3293  1.3293 0.7365 0.7365
 Queue1   Class2 0.5401 0.5556 2.2155  2.2155 0.2438 0.2438
 Queue2   Class1 2.2844 1.0000 3.1017  3.1017 0.7365 0.7365
 Queue2   Class2 0.9723 1.0000 3.9879  3.9879 0.2438 0.2438


/home/gcasale/Dropbox/code/line-dev.git/python/line_solver/api/pfqn/ncld.py:178: ComplexWarning: Casting complex values to real discards the imaginary part
  L = np.atleast_2d(np.asarray(L, dtype=float))


In [10]:
# Solve with MVA for load-dependent model
solver_mva_ld = MVA(ldmodel, method='exact')
lldAvgTableMVALD = solver_mva_ld.avg_table()

Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1 0.5467 0.5467 1.0000  1.0000 0.5467 0.5467
  Delay   Class2 0.3694 0.3694 2.0000  2.0000 0.1847 0.1847
 Queue1   Class1 0.8564 0.4739 1.5664  1.5664 0.5467 0.5467
 Queue1   Class2 0.4808 0.2713 2.6027  2.6027 0.1847 0.1847
 Queue2   Class1 2.5969 0.6941 4.7498  4.7498 0.5467 0.5467
 Queue2   Class2 1.1498 0.2980 6.2244  6.2244 0.1847 0.1847


In [11]:

solver_mva_qd = MVA(ldmodel, method='qd')
lldAvgTableQD = solver_mva_qd.avg_table()

Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
  Delay   Class1 0.1881 0.1881  1.0000  1.0000 0.1881 0.1881
  Delay   Class2 0.1415 0.1415  2.0000  2.0000 0.0708 0.0708
 Queue1   Class1 0.4571 0.2822  2.4296  2.4296 0.1881 0.1881
 Queue1   Class2 0.2866 0.1769  4.0493  4.0493 0.0708 0.0708
 Queue2   Class1 3.5429 0.6585 18.8309 18.8309 0.1881 0.1881
 Queue2   Class2 1.7134 0.3185 24.2112 24.2112 0.0708 0.0708


## Results Analysis

This example demonstrates the effect of load dependence on queue performance:

1. **Load-Independent Model**: Each queue has a fixed number of servers (3) regardless of population
2. **Load-Dependent Model**: Number of active servers depends on total population: min(population, 3)

The load dependence affects system performance by:
- Limiting the effective service capacity when population is low
- Providing full service capacity only when enough jobs are present
- This creates more realistic modeling of servers that scale with demand

Different solvers (CTMC, NC, MVA) provide various numerical methods for solving the same model, allowing comparison of accuracy and computational efficiency.